In [97]:
import pandas as pd
import pyspark.sql
import pyspark.sql.functions as sf

In [87]:
spark = pyspark.sql.SparkSession.Builder().getOrCreate()

# From Pandas to Spark

In this notebook we try to find patterns how common Pandas operations can be expressed in Spark. Since you should always avoid of switching back and forth between Spark and Pandas, you always should try to stay within a single framework. Actually the flexibility of Pandas is slightly bigger than that of Spark, but except for some specific exceptions you can do almost everything in Spark what you can do with Pandas, although the syntax and general approacj might differ.

## Fundamental Differences

Due to its inner design, Spark has some fundamental differences to Pandas. Specifically:

### Distributed processing
The huge selling point of Apache Spark is that it uses a distributed execution model running on multiple computers in a cluster, whereas Pandas is limited to a single Python process. While in Pandas your whole data set has to fit into memory, Spark can process data sets which are much bigger than the total amount of RAM of the whole compute cluster.

This non-functional high level difference has lead to a specific design of the implementation of Spark, which in turn has some very important implications when working with Spark. Some of the implications are formulated in the next items.

### Lazy evaluation
Spark does not execute any transformation when you specify it, but it chains together and optimizes all transformations whenever an action is started, for example to store or view the result of some transofrmations. This design is a very conscious decision of the Spark people (although an "eager" mode is planned for PySpark), which allows better optimizations since the execution of all transformations is delayed until the whole picture is clear to Spark. This approach allows rearranging transformations and pruning of columns thus greatly improving execution speed.

In general in Pandas you always work directly with the data, while in Spark you always transform the execution plan that will create some data for you.

### Immutable DataFrames
The whole core of Spark is developed in Scala, a object-oriented functional programming language. Again this was a very conscious design design of the founders of Spark. Functional programming in general prefers immutable objects over mutable ones. This is also true for the Spark API and helps to keep the Spark code simpler and more efficient. On the other hand, this also means that you cannot modify a data frame in place like you might be used to by Pandas. Every Spark transformation returns a new data frame (with some special exceptions, where some meta information is changed).

### No index
Pandas data frames always have an index (even if that is only the natural numbers), but Spark doesn't even have the concept of an index. Instead of an index, you might think about a primary key like you might know from relational databases.

### No single record access
By using the index, Pandas allows very efficient access to individual rows. This is completely impossible with Spark, since in Spark you primarily work with execution plans *representing* the data, but not with the data itself.

## TL;DR

Spark is much closer to a relational database with SQL than to Pandas. This requires some rethinking in some corner cases.

# 1. Reading Data

For working with data, we need to get some data first. Spark supports various file formats, we will use CSV in the following example.

The entrypoint for creating Spark objects is an object called `spark` which is provided in the notebook and read to use. We will read a file containing some informations on a couple of persons, which will serve as the basis for the next examples.

In [88]:
# Set the base directory according to your environment
basedir = "s3://dimajix-training/data/"

### Pandas

Pandas provides some simple methods for reading data from a local file system.

In [89]:
# YOUR CODE HERE

The Pandas data frame also has a specific schema, but the data types are focused on numerical types (i.e. there is no `string` type)

In [ ]:
# YOUR CODE HERE

### Spark

Spark of course also supports reading different file formats, ans also CSV. One important difference is that the data has to be accessible from all worker nodes in the cluster, and therefore needs to be stored on some *shared filesystem* like HDFS or S3.

In [33]:
# YOUR CODE HERE

Again the Spark data frame also has a schema, but it more closely resembles an SQL schema. It also directly supports `string` data types (but no other generic objects) and all columns also have a `nullable` property like relational databases also provide.

In [ ]:
# YOUR CODE HERE

# 2 Projections

The simplest thing to do is to create a new DataFrame with a subset of the available columns

### Pandas

A projection in Pandas is created by using the index operator (`[]`) where you can specify a list of desired column names.

In [ ]:
# YOUR CODE HERE

### Spark

In contrast to Pandas, Spark uses an already very powerful and flexible `select` mechanism, which can also perform only a simple projection. But the `select` method is the primary entry point for specifying transformations on a per record level in spark.

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

Specifically for the PySpark users there is also the possibility to perform a projection using the index operator (`[]`) like you know from Pandas. Technically this is still a `select` operation wrapped up in some syntactic sugar.

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

One noteable concept of Spark is that every transformation will return a new DataFrame. The original DataFrame remains unchanged. This is a deep architectural decision of Spark which simplifies parallel processing under the hood.

# 3 Simple Transformations

The `select` method actually accepts any *column* object. A *column* object conceptually represents a column in a DataFrame. The column may either refer directly to an existing column of the input DataFrame, or it may represent the result of a calculation or transformation of one or multiple columns of the input DataFrame. For example if we simply want to transform the name into upper case, we can do so by using a function `upper` provided by PySpark.

### Pandas

In [ ]:
result = # YOUR CODE HERE
result

### Spark

In [ ]:
result = # YOUR CODE HERE

result.toPandas()

## Adding Columns

Lets look at a differnt example where we want to create a new DataFrame with the appropriate salutation in front of the name.

### Pandas

In Pandas you can simply add a new column by assigning it via an index operator.

In [ ]:
def create_salutation(row):
    sex = row[0]
    name = row[1]
    if sex == 'male':
        return "Mr " + name
    else:
        return "Mrs " + name

# YOUR CODE HERE
result

### Spark

Spark does not allow modifying an existing data frame, instead a new data frame has to be created. The example above can be easily replicated via
* using the `withColumn` method to add a column to a data frame
* using SQL "control flow" provided via `when`

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

# 3. Filtering

*Filtering* denotes the process of keeping only rows which meet a certain filter criteria.

### Pandas

Filtering in Pandas can again be done via the index operator. The argument of the index operator essentially is an array containing boolean values for keeping or dropping a row from the data frame.

In [ ]:
result = # YOUR CODE HERE
result

### Spark

PySpark support two different approaches. The first approach specifies the filtering expression as a PySpark expression using columns:

In [ ]:
result = # YOUR CODE HERE
result.show()

In [ ]:
result = # YOUR CODE HERE
result.show()

The second approach simply uses a string containing an SQL expression:

In [ ]:
result = # YOUR CODE HERE
result.show()

# 4. Grouping & Aggregation

An important class of operation is grouping and aggregation, which is equivalnt to an SQL `SELECT aggregation GROUP BY grouping` statement. In PySpark, grouping and aggregation is always performed by first creating groups using `groupBy` immediately followed by aggregation expressions inside an `agg` method. (Actually there are also some predefined aggregations which can be used instead of `agg`, but they do not offer the flexiviliby which is required most of the time).

Note that in the `agg` method you only need to specify the aggregation expression, the grouping columns are added automatically by PySpark to the resulting DataFrame.

### Pandas

In [ ]:
result = persons_pd.groupby('sex').agg({
    'age':'mean',
    'height':['min','max']
})
result

### Spark

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

## Aggregation Functions

PySpark supports many aggregation functions, they can be found in the documentation at [PySpark Function Documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions). Aggregation functions are marked as such in the documentation, unfortunately there is no simple overview. Among common aggregation functions, there are for example:

* count
* sum
* avg
* corr
* first
* last

# 5. Joining Data

Every relation algebra also contains join operations which lets you combine multiple tables by a matching criterion. PySpark also supports joins of multiple DataFrames. In order to shed some light on that, we need a second DataFrame in addition to the `persons` DataFrame. Therefore we load some address data as follows:

In [ ]:
addresses = spark.read.json(basedir + "addresses.json")

addresses_pd = addresses.toPandas()
addresses_pd

Now that we have the `addresses` DataFrame, we want to combine it with the `persons` DataFrame such that the city of every person is added as a new column. This is achieved by the `join` method which essentially takes two parameters: The first parameter specifies the second DataFrame to join with, and the second parameter specifies the join condition. In this case we want to join all records, where the `name` column matches.

### Pandas

In [ ]:
result = # YOUR CODE HERE
result

### Spark

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

Let me make some relevant remarks:
* The resulting DataFrame now contains two `name` columns - one comes from the `persons` DataFrame, the other from the `addresses` DataFrame. Since the join condition could have used some more complex expression, this behaviour is only logical since PySpark cannot assume that all joins simply use directly some column value. For example we could also have transformed the column on the fly by converting the name to upper case directly inside the join condition.
* The result contains only persons where an address was found, although the original `persons` DataFrame contained more persons.
* There are no records of addresses without any person, although the `addresses` DataFrame contains information about some persons not available in the `persons` DataFrame.

So let us first address the first observation. We can easily get rid of the duplicated `name` column by either performing an explicit select of the desired columns, or by dropping the duplicate columns or by using a different join operator, which simply expects the list of join columns.

In [ ]:
result = # YOUR CODE HERE
result.toPandas()

Now let us explain the last two observations. These are due to the used join type, which was a so called *inner* join. In this case, only records with information from both DataFrames are included in the result.

In addition to the *inner* join, PySpark also supports some additional joins:
* *outer join* will contain records for all elements from both DataFrames. If either the left or right DataFrames doesn't contain any information, the result will contain `None` values (= `NULL` values) for the corresponding columns.
* In a *right join*, the second DataFrame (the right DataFrame) as specified as an argument is the leading element. The result will contain records for every record in that DataFrame.
* In a *left join*, the first DataFrame (the left DataFrame) as specified as the object iteself is the leading element. The result will contain records for every record in that DataFrame.

# 7. Reassembling data frames

One very flexible feature of Pandas is how you can construct or modify a data frame from columns. This is not easily possible with Spark. Let us have a look at an example

### Pandas

In [ ]:
df = persons_pd[["name", "age"]]
col = persons_pd["height"]

result = # YOUR CODE HERE
result

### Spark

The example above simply is not possible at all with Spark. Whenever you want to add a column with existing data, you have to perform a join. But since Spark does not have an explicit index, you also need to provide the join key in both data frames. So the best approximation would be:

In [ ]:
df = persons[["name", "age"]]
col = persons[["name", "height"]]

result = # YOUR CODE HERE
result.show()